In [1]:
import netgen.occ as occ
from ngsolve.webgui import Draw

r_coil = 0.025
y_coil = 0.05
r_outer = 0.1

c1 = occ.WorkPlane().Circle(r_outer).Face()
c2 = occ.WorkPlane().MoveTo(0,y_coil).Circle(r_coil).Face()
c3 = occ.WorkPlane().MoveTo(0,-y_coil).Circle(r_coil).Face()

full = occ.Glue([c1,c2,c3])

full.faces[0].name = 'stator'
full.faces[1].name = 'coil_plus'
full.faces[2].name = 'coil_minus'

full.edges[0].name = 'outer'

geoOCC = occ.OCCGeometry(full, dim = 2)

# Draw(full)

ngmesh = geoOCC.GenerateMesh()

import ngsolve as ngs
mesh = ngs.Mesh(ngmesh)
mesh.Curve(2)

mesh.Refine()
mesh.Refine()
mesh.Refine()

# Draw(mesh)

In [2]:
from bhdata import BHCurves
fun_dw  = BHCurves(3)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus"
nonlinear = "stator"

strom = 1e4

J = mesh.MaterialCF({'coil_plus': strom, 'coil_minus': -strom, 'stator': 0}, default = 0)
# Draw(J, mesh)

In [3]:
# print(HCurl.ndof)
# print(mesh.GetMaterials())

# H1 = ngs.H1(mesh, order = deg)

deg = 5
H1 = ngs.H1(mesh, order = deg, dirichlet = "outer")
u,v = H1.TnT()

rot = ngs.CF((0,1,-1,0), dims=(2,2))
def curl2d(a):
    return rot*ngs.grad(a)

# Nonlinear:

maxit = 100000
tol2 = 1e-8
regb = 1e-12


A = ngs.GridFunction(H1)
B = curl2d(A)
normB = ngs.sqrt(B*B + regb)

ir = ngs.IntegrationRule(ngs.fem.ET.TRIG, order = 2*deg)


cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B).Compile()

def fun_W():
    # with ngs.TaskManager(): res = ngs.Integrate(cf_energy - curl2d(A)*Hs, mesh, order = 2*deg)
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - A*J, mesh, order = 2*deg)
    return res


cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0).Compile()

rhs = ngs.LinearForm(H1)
# rhs += ngs.SymbolicLFI(cf_rhs*B*curl2d(v) - ngs.curl(Hs)*v, intrule = ir)
rhs += ngs.SymbolicLFI(cf_rhs*B*curl2d(v) - J*v, intrule = ir)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs


Id = ngs.CF((1,0,
             0,1), dims = (2,2))

BBt = ngs.CF((B[0]*B[0], B[0]*B[1],
              B[1]*B[0], B[1]*B[1]), dims = (2,2))


fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id).Compile()

# ir = ngs.IntegrationRule(points = [(1/3,1/3)], weights = [1/2])
K_iter = ngs.BilinearForm(H1)
# K_iter += ngs.SymbolicBFI(cf_iter*curl2d(u)*curl2d(v))
K_iter += ngs.SymbolicBFI(cf_iter*curl2d(u)*curl2d(v), intrule = ir)
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter

In [4]:
from ttictoc import tic, toc
from ngsolve.krylovspace import CGSolver

print("Using 2D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,H1.ndof, "DOFs.\n ")

with ngs.TaskManager(): A.Set(ngs.CF((0)))

du = ngs.GridFunction(H1)
uo = ngs.GridFunction(H1)
wo = 1e12

for it in range(1,maxit+1):
    tic()
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()
    # du.vec.data = da.mat.Inverse(H1.FreeDofs(), inverse="sparsecholesky") * dw.vec 
    # iterativeSolver = CGSolver(K_iter.mat, freedofs = H1.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
    with ngs.TaskManager():
        du.vec.data = da.mat.Inverse(H1.FreeDofs(), inverse = "sparsecholesky") * dw.vec
        # iterativeSolver = CGSolver(K_iter.mat, freedofs = H1.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
        # iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-8,  maxiter = maxit)
        # du.vec.data = iterativeSolver * dw.vec
    
    # if len(iterativeSolver.residuals) == maxit: print("... Failure!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()

    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm
    
    # wn = 1e12
    if abs(wo-w)/abs(w) < tol2:
    # if abs(wn-w) < tol2:
    # if nrm/nrm0 < tol2:
        print("converged to desired tolerance")
        break
    # elif abs(wo-w)< tol2*1e-2:
    #     # print(abs(wo-w),abs(w),alpha,nrm,nrm0)
    #     print("stopped early due to stagnation | energy= %.10f" %w)
    #     # print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
    #     break
    else:
        # linesearch
        uo.vec.data = A.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            A.vec.data -= alpha*du.vec.data
            wn = fun_W()
            if wn < w - alpha*0.01*nrm:
                print("Iter: %2d | assem : %.2fs | CG took %.2fs | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,alpha,w,nrm/nrm0))
                break
            else:
                alpha = alpha/2
                A.vec.data = uo.vec.data


Draw(ngs.grad(A), mesh)

Using 2D mesh with ne= 12224 elements and nv= 6197 points and  153221 DOFs.
 
Iter:  1 | assem : 0.10s | CG took 0.54s | alpha : 0.00 | energy = 0.0000000000 | relres = 1.00e+00 |
Iter:  2 | assem : 0.06s | CG took 0.54s | alpha : 1.00 | energy = -0.0280226271 | relres = 2.13e-05 |
Iter:  3 | assem : 0.06s | CG took 0.54s | alpha : 1.00 | energy = -0.0378226294 | relres = 2.55e-07 |
Iter:  4 | assem : 0.07s | CG took 0.54s | alpha : 1.00 | energy = -0.0379495679 | relres = 3.67e-11 |
Iter:  5 | assem : 0.06s | CG took 0.53s | alpha : 1.00 | energy = -0.0379495861 | relres = 7.76e-17 |
converged to desired tolerance


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene